In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_table('/data2/deepLN/table.kor_sfari_mssng.DNV_annotated.coding_combinations.20250226.tsv.gz')

In [3]:
df.rename(columns=lambda x: x.replace("is_", "") if "is_" in x else x, inplace=True)

In [4]:
df

,SAMPLE,variant,gene_id,gene_name,coding,MGE.dev,CGE.dev,AST,L2.3,L4,...,SP,DDD285,CHD8Common,FMRPDarnell,ASD185,PTV,modMIS,dMIS,promoter,UTR
0,1-0007-003,chr10:64274121:T:C,ENSG00000235489,DBF4P1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
1,1-0007-003,chr11:133580639:G:C,ENSG00000183715,OPCML,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
2,1-0007-003,chr11:21439019:C:T,ENSG00000252816,RNA5SP337,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
3,1-0007-003,chr11:43777119:G:A,ENSG00000283483,ENSG00000283483,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
4,1-0007-003,chr11:46734475:TATCA:T,ENSG00000180210,F2,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967201,SSC12972,chr8:40970746:A:AC,ENSG00000206867,RNU6-356P,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
967202,SSC12972,chr8:4721806:A:C,ENSG00000254244,PAICSP4,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
967203,SSC12972,chr8:48224849:G:A,ENSG00000253843,ENSG00000253843,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0
967204,SSC12972,chr8:73900077:G:T,ENSG00000251633,GYG1P1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0


In [5]:
# Define column groups
celltype_cols = ['MGE.dev', 'CGE.dev', 'AST', 'L2.3', 'L4', 'L5', 'L5.6.IT', 'L6', 'MG', 'OL', 'END', 'PER', 'SP']
asd_gene_cols = ['DDD285', 'CHD8Common', 'FMRPDarnell', 'ASD185']
varianttype_cols = ['PTV', 'modMIS', 'dMIS', 'promoter', 'UTR']

In [6]:
# Function to generate combinations
def generate_combinations(row):
    # A Condition: Cell type
    A_matches = [col for col in celltype_cols if row[col] == 1] or ["None"]
    
    # B Condition: ASD gene
    B_matches = [col for col in asd_gene_cols if row[col] == 1] or ["None"]
    
    # C Condition: Variant type
    C_matches = [col for col in varianttype_cols if row[col] == 1] or ["None"]
    
    # Generate all combinations
    combinations = [f"{A}_{B}_{C}" for A in A_matches for B in B_matches for C in C_matches]
    return combinations

In [7]:
# Apply function to generate combinations for each row
df['combinations'] = df.apply(generate_combinations, axis=1)

In [8]:
# Get all unique combinations
all_combinations = set(comb for row_combs in df['combinations'] for comb in row_combs)

In [9]:
temp_df = pd.DataFrame(0, index=df.index, columns=list(all_combinations))

In [10]:
# Fill in the new combination columns
for idx, row in df.iterrows():
    for comb in row['combinations']:
        temp_df.at[idx, comb] = 1

In [11]:
# Merge the temporary dataframe into the original dataframe
df = pd.concat([df, temp_df], axis=1)

In [12]:
df

,SAMPLE,variant,gene_id,gene_name,coding,MGE.dev,CGE.dev,AST,L2.3,L4,...,CGE.dev_ASD185_PTV,SP_CHD8Common_promoter,OL_ASD185_modMIS,L5.6.IT_CHD8Common_promoter,L5.6.IT_None_modMIS,L4_None_modMIS,L6_CHD8Common_dMIS,L6_CHD8Common_promoter,L6_None_None,END_ASD185_PTV
0,1-0007-003,chr10:64274121:T:C,ENSG00000235489,DBF4P1,0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1-0007-003,chr11:133580639:G:C,ENSG00000183715,OPCML,0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1-0007-003,chr11:21439019:C:T,ENSG00000252816,RNA5SP337,0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,1-0007-003,chr11:43777119:G:A,ENSG00000283483,ENSG00000283483,0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,1-0007-003,chr11:46734475:TATCA:T,ENSG00000180210,F2,0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967201,SSC12972,chr8:40970746:A:AC,ENSG00000206867,RNU6-356P,0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
967202,SSC12972,chr8:4721806:A:C,ENSG00000254244,PAICSP4,0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
967203,SSC12972,chr8:48224849:G:A,ENSG00000253843,ENSG00000253843,0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
967204,SSC12972,chr8:73900077:G:T,ENSG00000251633,GYG1P1,0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Drop the temporary 'combinations' column
df = df.drop(columns=['combinations'])

In [14]:
# Define columns to be removed
columns_to_remove = celltype_cols + asd_gene_cols + varianttype_cols + ['gene_id', 'gene_name']

# Drop these columns from the dataframe
df = df.drop(columns=columns_to_remove)

# Display the final dataframe
print(df.head())

       SAMPLE                 variant  coding  L5_ASD185_modMIS  \
0  1-0007-003      chr10:64274121:T:C       0                 0   
1  1-0007-003     chr11:133580639:G:C       0                 0   
2  1-0007-003      chr11:21439019:C:T       0                 0   
3  1-0007-003      chr11:43777119:G:A       0                 0   
4  1-0007-003  chr11:46734475:TATCA:T       0                 0   

   None_DDD285_promoter  None_CHD8Common_dMIS  PER_None_modMIS  \
0                     0                     0                0   
1                     0                     0                0   
2                     0                     0                0   
3                     0                     0                0   
4                     0                     0                0   

   MGE.dev_DDD285_None  L4_DDD285_None  L5.6.IT_ASD185_dMIS  ...  \
0                    0               0                    0  ...   
1                    0               0                    0  ...

In [15]:
df = df.drop(columns=["None_None_None"])

In [16]:
df

,SAMPLE,variant,coding,L5_ASD185_modMIS,None_DDD285_promoter,None_CHD8Common_dMIS,PER_None_modMIS,MGE.dev_DDD285_None,L4_DDD285_None,L5.6.IT_ASD185_dMIS,...,CGE.dev_ASD185_PTV,SP_CHD8Common_promoter,OL_ASD185_modMIS,L5.6.IT_CHD8Common_promoter,L5.6.IT_None_modMIS,L4_None_modMIS,L6_CHD8Common_dMIS,L6_CHD8Common_promoter,L6_None_None,END_ASD185_PTV
0,1-0007-003,chr10:64274121:T:C,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1-0007-003,chr11:133580639:G:C,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1-0007-003,chr11:21439019:C:T,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1-0007-003,chr11:43777119:G:A,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1-0007-003,chr11:46734475:TATCA:T,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967201,SSC12972,chr8:40970746:A:AC,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
967202,SSC12972,chr8:4721806:A:C,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
967203,SSC12972,chr8:48224849:G:A,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
967204,SSC12972,chr8:73900077:G:T,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df.to_csv("/data2/deepLN/kor_sfari_mssng.DNV_annotated.coding_combinations.agg.20250227.tsv.gz", sep="\t", index=False, compression="gzip")

In [18]:
# 전체 데이터에 대해 SAMPLE별로 합계
counts_by_sample_all = df.drop(columns=['variant', 'coding']).groupby('SAMPLE').sum()

# 전체에 대해 비율 계산
proportions_all = df.drop(columns=['variant', 'coding']).groupby('SAMPLE').apply(lambda x: (x == 1).mean())
proportions_all = proportions_all.add_prefix('prop_')  # 비율에 'prop_' 접두어 추가

# coding == 1인 행만 필터링한 후 SAMPLE별로 합계
counts_by_sample_coding_1 = df[df['coding'] == 1].drop(columns=['variant', 'coding']).groupby('SAMPLE').sum()
counts_by_sample_coding_1 = counts_by_sample_coding_1.add_prefix('cd_')  # 열 이름에 'cd_' 접두어 추가

# coding == 1에 대해 비율 계산
proportions_coding_1 = df[df['coding'] == 1].drop(columns=['variant', 'coding']).groupby('SAMPLE').apply(lambda x: (x == 1).mean())
proportions_coding_1 = proportions_coding_1.add_prefix('prop_cd_')  # 비율에 'prop_cd_' 접두어 추가

# coding == 0인 행만 필터링한 후 SAMPLE별로 합계
counts_by_sample_coding_0 = df[df['coding'] == 0].drop(columns=['variant', 'coding']).groupby('SAMPLE').sum()
counts_by_sample_coding_0 = counts_by_sample_coding_0.add_prefix('nc_')  # 열 이름에 'nc_' 접두어 추가

# coding == 0에 대해 비율 계산
proportions_coding_0 = df[df['coding'] == 0].drop(columns=['variant', 'coding']).groupby('SAMPLE').apply(lambda x: (x == 1).mean())
proportions_coding_0 = proportions_coding_0.add_prefix('prop_nc_')  # 비율에 'prop_nc_' 접두어 추가

# 전체, coding == 1, coding == 0 데이터프레임을 합침
counts_combined = pd.concat([counts_by_sample_all, counts_by_sample_coding_1, counts_by_sample_coding_0], axis=1)

# 비율과 raw count 합침
counts_combined_with_proportions = pd.concat([counts_combined, proportions_all, proportions_coding_1, proportions_coding_0], axis=1)

/tmp/ipykernel_903444/1570361289.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  proportions_all = df.drop(columns=['variant', 'coding']).groupby('SAMPLE').apply(lambda x: (x == 1).mean())
/tmp/ipykernel_903444/1570361289.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  proportions_coding_1 = df[df['coding'] == 1].drop(columns=['variant', 'coding']).groupby('SAMPLE').apply(lambda x: (x == 1).mean())

In [19]:
counts_combined_with_proportions

,L5_ASD185_modMIS,None_DDD285_promoter,None_CHD8Common_dMIS,PER_None_modMIS,MGE.dev_DDD285_None,L4_DDD285_None,L5.6.IT_ASD185_dMIS,None_ASD185_PTV,L4_None_promoter,AST_FMRPDarnell_UTR,...,prop_nc_CGE.dev_ASD185_PTV,prop_nc_SP_CHD8Common_promoter,prop_nc_OL_ASD185_modMIS,prop_nc_L5.6.IT_CHD8Common_promoter,prop_nc_L5.6.IT_None_modMIS,prop_nc_L4_None_modMIS,prop_nc_L6_CHD8Common_dMIS,prop_nc_L6_CHD8Common_promoter,prop_nc_L6_None_None,prop_nc_END_ASD185_PTV
SAMPLE,,,,,,,,,,,,,,,,,,,,,
1-0007-003,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013889,0.0
1-0009-003,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1-0009-004,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1-0009-005,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1-0014-003,0,0,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SSC12962,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
SSC12964,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
SSC12967,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [20]:
counts_combined_with_proportions = counts_combined_with_proportions.drop(columns=[col for col in counts_combined_with_proportions.columns if 'SAMPLE' in col])

In [21]:
counts_combined_with_proportions = counts_combined_with_proportions.reset_index()

In [22]:
counts_combined_with_proportions = counts_combined_with_proportions.fillna(0)

In [23]:
sample_columns = [col for col in counts_combined_with_proportions.columns if 'SAMPLE' in col]
sample_columns

['SAMPLE']

In [24]:
counts_combined_with_proportions

,SAMPLE,L5_ASD185_modMIS,None_DDD285_promoter,None_CHD8Common_dMIS,PER_None_modMIS,MGE.dev_DDD285_None,L4_DDD285_None,L5.6.IT_ASD185_dMIS,None_ASD185_PTV,L4_None_promoter,...,prop_nc_CGE.dev_ASD185_PTV,prop_nc_SP_CHD8Common_promoter,prop_nc_OL_ASD185_modMIS,prop_nc_L5.6.IT_CHD8Common_promoter,prop_nc_L5.6.IT_None_modMIS,prop_nc_L4_None_modMIS,prop_nc_L6_CHD8Common_dMIS,prop_nc_L6_CHD8Common_promoter,prop_nc_L6_None_None,prop_nc_END_ASD185_PTV
0,1-0007-003,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.013889,0.0
1,1-0009-003,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,1-0009-004,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,1-0009-005,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,1-0014-003,0,0,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14601,SSC12962,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
14602,SSC12964,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
14603,SSC12967,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
14604,SSC12971,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [25]:
counts_combined_with_proportions.to_csv("/data2/deepLN/kor_sfari_mssng.coding_combinations.agg_by_sample.20250227.tsv.gz", sep="\t", index=False, compression="gzip")